<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete EBS (Elastic Block Storage) Volumes associated with stopped EC2 instances</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-EBS-Volumes-Attached-To-Stopped-Instances"><u>Delete EBS Volumes Attached To Stopped Instances</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Get volumes for stopped instances</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete EBS volumes</a></p>

In [ ]:
if volume_ids and not region:
    raise SystemExit("Provide a region for the EBS Volume IDs!")

<h3 id="List-all-AWS-Regions">List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 --region us-west-2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')
task.configure(outputName="region")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Get-volumes-for-stopped-instances"><a id="1" target="_self" rel="nofollow"></a>Get volumes for stopped instances</h3>
<p>Using unSkript's Get Stopped Instances EBS volumes action, we will find volumes which are associated with stopped instances.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>stopped_instances_volumes</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_get_stopped_instance_volumes_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_stopped_instance_volumes(handle, region: str = "") -> Tuple:
    """aws_get_stopped_instance_volumes Returns an array of volumes.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region to filter instances.

        :rtype: Array of volumes.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=reg)
            res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
            for reservation in res:
                for instance in reservation['Instances']:
                    if instance['State']['Name'] == 'stopped':
                        block_device_mappings = instance['BlockDeviceMappings']
                        for mapping in block_device_mappings:
                            if 'Ebs' in mapping:
                                ebs_volume = {}
                                volume_id = mapping['Ebs']['VolumeId']
                                ebs_volume["volume_id"] = volume_id
                                ebs_volume["region"] = reg
                                result.append(ebs_volume)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not volume_ids",
    "condition_result": true
    }''')
task.configure(outputName="stopped_instances_volumes")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_stopped_instance_volumes, lego_printer=aws_get_stopped_instance_volumes_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Stopped-Instance-Volumes">Create List of Stopped Instance Volumes</h3>
<p>This action filters regions that have no volumes associated with stopped instances and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_stopped_instances_volumes</code></p>
</blockquote>

In [ ]:
all_stopped_instances_volumes = []
dummy = []
try:
    for reg,res in stopped_instances_volumes.items():
        if res[0]==False:
            if len(res[1])!=0:
                dummy = res[1]
                for x in dummy:
                    all_stopped_instances_volumes.append(x)
except Exception as e:
    for vol_id in volume_ids:
        data_dict = {}
        data_dict["region"] = region[0]
        data_dict["volume_id"] = vol_id
        all_stopped_instances_volumes.append(data_dict)
print(all_stopped_instances_volumes)
task.configure(outputName="all_stopped_instances_volumes")

<h3 id="Delete-Low-Usage-EBS-Volumes"><a id="2" target="_self" rel="nofollow"></a>Delete EBS Volumes</h3>
<p>This action deletes volumes found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>volume_id, region</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2022 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_delete_volume_by_id_printer(output):
    if output is None:
        return
    pprint.pprint({"Output": output})


@beartype
def aws_delete_volume_by_id(handle, volume_id: str, region: str) -> str:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :type volume_id: string
        :param volume_id: Volume ID needed to delete particular volume.

        :rtype: Result of the API in the List form.
    """
    result = []

    ec2Client = handle.client('ec2',region_name=region)

    # Adding logic for deletion criteria
    try:
        response = ec2Client.delete_volume(VolumeId=volume_id,)
        result.append(response)
    except Exception as e:
        result.append(e)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "volume_id": "iter.get(\\"volume_id\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_stopped_instances_volumes",
    "iter_parameter": ["volume_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_stopped_instances_volumes)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_volume_by_id, lego_printer=aws_delete_volume_by_id_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to get EBS volumes attached to EC2 instances that have been stopped and delete them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>